In [1]:
%run ../../Utils/yp_utils.py

# Initial setup

In [2]:
paper_pmid = 30646537
paper_name = 'corban_gueven_2019' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [8]:
original_data = pd.read_excel('raw_data/marinedrugs-17-00054-s001.xlsx', sheet_name='Downregulated only', skiprows=1)

In [9]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 115 x 7


In [10]:
original_data.head()

,ORF NAME,Gene Name,Cellular Process,Predicted Protein Function,Unnamed: 4,Unnamed: 5,Growth Inhibition (Fold Change)
0,YGL202W,ARO8,Amino acid biosynthesis,Aromatic aminotransferase I,NaN,NaN,2.941176
1,YGL148W,ARO2,Amino acid biosynthesis,Bifunctional chorismate synthase and flavin re...,NaN,NaN,3.846154
2,YER055C,HIS1,Amino acid biosynthesis,ATP phosphoribosyltransferase,NaN,NaN,1.886792
3,YMR123W,PKR1,ATP synthesis,V-ATPase assembly factor,NaN,NaN,1.851852
4,YJR096W,YJR096W,Carbohydrate metabolism,Putative xylose and arabinose reductase,NaN,NaN,2.222222


In [11]:
original_data['orf'] = original_data['ORF NAME'].astype(str)

In [12]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [13]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [14]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

Empty DataFrame
Columns: [ORF NAME, Gene Name, Cellular Process, Predicted Protein Function, Unnamed: 4, Unnamed: 5, Growth Inhibition (Fold Change), orf]
Index: []


In [15]:
original_data['data'] = -pd.to_numeric(original_data['Growth Inhibition (Fold Change)'], errors='coerce')

In [16]:
original_data.set_index('orf', inplace=True)

In [17]:
original_data = original_data[['data']].copy()

In [18]:
original_data = original_data.groupby(original_data.index).mean()

In [19]:
original_data.shape

(115, 1)

# Prepare the final dataset

In [20]:
data = original_data.copy()

In [21]:
dataset_ids = [16713]
datasets = datasets.reindex(index=dataset_ids)

In [22]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [23]:
data.head()

dataset_id,16713
data_type,value
orf,
YAL016W,-2.941176
YAL035W,-2.173913
YBR081C,-3.846154
YBR098W,-5.000000
YBR103W,-3.703704


## Subset to the genes currently in SGD

In [24]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 0


In [25]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,16713
,data_type,value
gene_id,orf,
14,YAL016W,-2.941176
33,YAL035W,-2.173913
276,YBR081C,-3.846154
293,YBR098W,-5.000000
297,YBR103W,-3.703704


# Normalize

In [26]:
data_norm = normalize_phenotypic_scores(data, has_tested=False)

In [27]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [28]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

dataset_id          16713          
data_type           value    valuez
gene_id orf                        
14      YAL016W -2.941176 -5.290517
33      YAL035W -2.173913 -3.910382
276     YBR081C -3.846154 -6.918368
293     YBR098W -5.000000 -8.993878
297     YBR103W -3.703704 -6.662132

# Print out

In [29]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [30]:
from IO.save_data_to_db3 import *

In [31]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/1 [00:00<?, ?it/s]

Deleting all datasets for PMID 30646537...
Inserting the new data...


100%|██████████| 1/1 [00:00<00:00,  3.17it/s]

Updating the data_modified_on field...
